In [6]:
!pip install -U transformers


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizerBase,
    pipeline,
)
import os

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B-Instruct")

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
all_datasets = ['DFINAL_VTrain.csv', 'DFINAL_VTest.csv']

In [4]:
for dataset in all_datasets:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: eval(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    new_path = os.path.join('../../data/training_data', dataset.split('.')[0] + '_qwen-coder.csv')
    df.to_csv(new_path, index=False)

In [11]:
df.head()

,Unnamed: 0,game,benchmark_version,game_id,model,experiment,episode,Aborted,Lose,Success,chat,text
0,0,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...",<|begin_of_text|><|start_header_id|>system<|en...
1,1,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...",<|begin_of_text|><|start_header_id|>system<|en...
2,2,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...",<|begin_of_text|><|start_header_id|>system<|en...
3,3,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...",<|begin_of_text|><|start_header_id|>system<|en...
4,4,imagegame,v1.6,0,o1-preview-2024-09-12-t0.0--o1-preview-2024-09...,1_random_grids,episode_0,0,0,1,"[{'role': 'user', 'content': 'Let us do the fo...",<|begin_of_text|><|start_header_id|>system<|en...


In [24]:
df.text.iloc[0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nLet us draw something together. There is an empty grid with a size 5 by 5, like so:\n\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n\nI will give you instructions like "put an X in the top left", and you return the grid by applying the given instruction, like so:\n\nInstruction: put an X in the top left\n\nX ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\n\nOr for another instruction such as "fill the fifth column with T", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:\n\nInstruction: fill the fifth column with T\n\nX ▢ ▢ ▢ T\n▢ ▢ ▢ ▢ T\n▢ ▢ ▢ ▢ T\n▢ ▢ ▢ ▢ T\n▢ ▢ ▢ ▢ T\n\nOr for another instruction such as "fill the fourth column second row with P", you return the updated grid by applying the given instruction in all pla

In [3]:
import pandas as pd
import os
from datasets import Dataset, load_dataset
df = pd.read_csv(os.path.join('../../data/training_data', 'DFINAL_qwen-coder.csv'))
ds: Dataset = Dataset.from_pandas(df=df)

In [36]:
for i,c in enumerate(list(df.chat)):
    if not isinstance(c, str):
        print(c, i)

In [55]:
for d in ds["chat"]:
    if not isinstance(d, str):
        print(d)

In [7]:
print(df.text.iloc[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Let us draw something together. There is an empty grid with a size 5 by 5, like so:

▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

I will give you instructions like "put an X in the top left", and you return the grid by applying the given instruction, like so:

Instruction: put an X in the top left

X ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢
▢ ▢ ▢ ▢ ▢

Or for another instruction such as "fill the fifth column with T", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fifth column with T

X ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T
▢ ▢ ▢ ▢ T

Or for another instruction such as "fill the fourth column second row with P", you return the updated grid by applying the given instruction in all places that the command corresponds to, like so:

Instruction: fill the fourth column second row with P

X ▢ ▢ ▢ T
▢ ▢ ▢ P T
▢ ▢ ▢ ▢ T
▢ ▢ ▢